# Initial Setup 

Domino model monitoring involves connecting your model and external data sources to Domino Model Monitoring. This starter notebook walks through setting up the components of Domino Model Monitor that will be used in the "2_External_DMM_Quickstart", "3_Integrated_DMM_Quickstart" and "4_Custom_Metrics_Quickstart.ipynb" notebooks.

This notebook also saves the name and configs for your datasource in a config file under your Project Artifacts. This config file will streamline automated scoring & ground truth data checks later on, without committing any sensitive information to a git repo.

## Domino API Key

The Quickstart notebooks have examples for interacting with Domino Model Monitoring via API (for example, registering a new datasource in Domino Model Monitoring's API).

Your Domino User API key is stored by default as the user environment variable "DOMINO_USER_API_KEY". Treat your API key like a sensitive password, and do not save it anywhere in this notebook, as it could be committed to an external git service if you sync your workspace. Always access it using the user environment variable.

Authenticating with your Domino API key is documented here:

https://docs.dominodatalab.com/en/latest/user_guide/40b91f/domino-api-authentication/#_authenticate_with_an_api_key

**Note on your Domino API Key:** Note that Domino Model Monitoring API keys have been deprecated, there is now only one API key for your whole Domino account.

*If you need to update or rotate your Domino API key, go to the Domino Workbench, then open up your User Account setting in the lower left. Regenerate your API key, save it securely, then also save to your Domino account as a User Environment Variable.*

*Your Workspace will not yet know about your new User Environment Variable. Save your notebook, then save and restart your Workspace to make your workspace aware of the new environment variable. Also be sure to update the references to the API key in these notebooks to reference your new variable name.*

In [14]:
# Verify that you can access your API key. 
import os

my_key = os.environ['DOMINO_USER_API_KEY']

## Register your Domino Model Monitoring Datasource

Domino requires an external data source for external models. For integrated models, you only need a Monitoring Data Source if you are ingesting ground truth labels. A datasource can be set up in the Domino Model monitoring UI, or via the Domino Model Monitoring API.

Both Quickstart Notebooks use an AWS s3 bucket for their Domino Model Monitoring datasource, but other data sources such as Snowflake or ADLS can be used. Domino Model Monitoring data sources are documented here:

https://docs.dominodatalab.com/en/latest/user_guide/8c7833/connect-a-data-source/

The Monitoring Data Sources are registered independently of the data sources used in Domino Workbench. The Quickstart Examples pass data from this Domino Project in the Domino Workbench to Domino Model Monitoring through the external datasource, so we will need to add the same external data source to both Domino Model Monitoring and this Project.

### Steps

(1) Set up your external storage outside of Domino (in AWS, Azure, GCP, Snowflake or HDFS). Note that to set up a Monitoring Datasource, you will need your key / password for that datsource. Note the location for monitoring files in your data source (e.g. bucket name, database name, container etc.).

*If you do not have those credentials, contact your Admin and ask them to add the datasource to Domino Model Monitoring and grant you access to the data source in the Domino Workbench using a service account.*

(2) Add the datasource to Domino Model Monitoring. Note the model monitoring "Data Source Name" when you create it, this will be used to access the datasource.

<img src="readme_images/Add_DMM_Datasource.png" width="600" height="600"> 



(3) Add the same datasource to this Project in the workbench. Note the workbench data source name in the data tab on the left.



<img src="readme_images/Add_DMM_Workbench.png" width="600" height="600"> 




## Save Dataset names and API reference in a config file

To reference the names of the data sources created above, as well as your Domino URL.

We'll save this in a configuration yaml file in our Project Artifacts. The other Quick Setup notebook will reference these configs.

Update each of the fields below, then run the cell, and commit your workspace. A new file called "DMM_config.yaml" should appear in your Project Artifacts folder.

In [17]:
import yaml

# UPDATE 1: Set the name for your new datasource in Domino Model Monitoring

DMM_datasource_name = 'monitoring-workshop'

# UPDATE 2: Set the name for your new data source in this Project (the name in the Data tab to the left)

workbench_datasource_name = 'monitoring-workshop'

# UPDATE 3: Datasource path. This will vary depending on the datasource type used. 
# For this s3 example, it is the bucket the monitoring files will be stored in.

datasource_location = "monitoring-workshop"

# UPDATE 4: Your Domino URL for API calls don't include the 'http://' or the trailing '/'

your_domino_url = "monitoring-url"

# UPDATE 5: The type of datasource (s3, ADLS, Snowflake etc). Names are documented here:
# https://docs.dominodatalab.com/en/latest/user_guide/8c7833/connect-a-data-source/

datasource_type = "s3"

DMM_configs = {
        'DMM_datasource_name': DMM_datasource_name,
        'workbench_datasource_name': workbench_datasource_name,
        'url': your_domino_url,
        'datasource':{
            'type': datasource_type,
            'location': datasource_location     
        },
        'integrated_model_url': '_', # For 3_Integrated_DMM_Quickstart
        'integrated_model_auth': '_', # For 3_Integrated_DMM_Quickstart
        'integrated_model_id': '_' # For 3_Integrated_DMM_Quickstart
    }


# Create a yaml file in artifacts, and save configs above.
!touch /mnt/artifacts/DMM_config.yaml

with open("/mnt/artifacts/DMM_config.yaml", 'w') as yamlfile:
    data = yaml.dump(DMM_configs, yamlfile)
    print("Write to config file successful")

print('Remember to save this notebook & sync your Workspace!')

Write to config file successful
Remember to save this notebook & sync your Workspace!


### (Optional) Register Datasource via API

You can register a datasource in Domino Model Monitoring using the UI (shown above) or using an API call.  The following example uses an AWS s3 bucket.

The APIs for other data source types are documented here:

https://docs.dominodatalab.com/en/latest/api_guide/f31cde/model-monitoring-api-reference/#_s3Request

Like in the UI, registering a data source via API requires your secret key or password, which should be securely stored as user environment variables. 

Go to the Domino Workbench, then open up your User Account setting in the lower left. Save the required keys for your datasource as User Environment Variables. In this example, I’ve called the s3 keys 'AWS_ACCESS_KEY_ID' and 'AWS_SECRET_ACCESS_KEY'.

Your Workspace will not yet know about your new User Environment Variable. Save your notebook, then save and restart your Workspace to make your workspace aware of the new user environment variable. Note that these keys will not be available to Project collaborators, only you.

In [ ]:
# s3 Example

import json
import requests

# UPDATE 1: Set the name for your new datasource in Domino Model Monitoring
DMM_datasource_name = 'API-Datasource'

# UPDATE 2: Cloud region, needed for s3 bucket registration
S3_Region = "us-west-2"

# AWS Keys, saved as User Environment Variables
AWS_Access_Key = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_Secret_Key = os.environ.get("AWS_SECRET_ACCESS_KEY")

# Fetch configs
with open("/mnt/artifacts/DMM_config.yaml") as yamlfile:
    config = yaml.safe_load(yamlfile)

datasource_url = "https://{}/model-monitor/v2/api/datasource".format(config['url'])

# Set up call headers
headers = {
           'X-Domino-Api-Key': os.environ['DOMINO_USER_API_KEY'],
           'Content-Type': 'application/json'
          }

data_source_request = {
    "name": DMM_datasource_name,
    "type": config['datasource']['type'],
    "config" : {
        "bucket": config['datasource']['location'],
        "region": S3_Region,
        "instance_role" : False,
        "access_key": AWS_Access_Key,
        "secret_key": AWS_Secret_Key
    }
}

# Make api call
ground_truth_response = requests.request("PUT", datasource_url, headers=headers, data = json.dumps(data_source_request))
 
# Print response
print(ground_truth_response.text.encode('utf8'))

# Finally, add datasource-specific configs to the yaml file. These will vary depending on your datasource type.
config['datasource']['S3_Region'] = S3_Region
config['DMM_datasource_name'] = DMM_datasource_name

with open("/mnt/artifacts/DMM_config.yaml", "w") as yamlfile:
    config = yaml.dump(
        config, stream=yamlfile, default_flow_style=False, sort_keys=False
    )

print('DONE!')

print('Remember to save this notebook & sync your Workspace!')